In [60]:
import os

In [61]:
%pwd

'D:\\Text-Summarizer'

In [62]:
os.chdir("D:\Text-Summarizer")
%pwd

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\bansa\AppData\Local\Temp\ipykernel_8820\1982117806.py:1: SyntaxWarning: invalid escape sequence '\T'
  os.chdir("D:\Text-Summarizer")


'D:\\Text-Summarizer'

In [63]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [64]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config
    

In [66]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [67]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def unzip_file(self):

        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [73]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_file()
except Exception as e:
    raise e

2025-03-08 17:03:26,869: INFO: common: yaml file :config\config.yaml read successfully
2025-03-08 17:03:26,873: INFO: common: yaml file :params.yaml read successfully
2025-03-08 17:03:26,877: INFO: common: Created directory: artifacts
2025-03-08 17:03:26,881: INFO: common: Created directory: artifacts/data_ingestion


2025-03-08 17:03:31,528: INFO: 3735868675: artifacts/data_ingestion/summarizer-data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 64B5:11327:30D088:814F4F:67CC2B07
Accept-Ranges: bytes
Date: Sat, 08 Mar 2025 11:33:28 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10237-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1741433608.003159,VS0,VE880
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 2b14ed2b7cdf512363347ce46b52594a539b0a06
Expires: Sat, 08 Mar 2025 11:38:28 GMT
Source-Age: 1


